# Table tennis simulation

This example shows the usage of `Simulate.jl` with event driven state machines.

We implement players as timed state machines and thus need definitions of states and events and some data describing the players:

In [1]:
using Simulate, Random, Printf

abstract type PState end
struct Idle <: PState end
struct Wait <: PState end
struct Unalert <: PState end

abstract type PEvent end
struct Start <: PEvent end
struct Serve <: PEvent end
struct Return <: PEvent end
struct Miss <: PEvent end

mutable struct Player
    name::AbstractString
    opp::Union{Number,Player}
    state::PState
    accuracy::Float64
    attentiveness::Float64
    score::Int64

    Player(name, acc, att) = new(name, 0, Idle(), acc, att, 0)
end

Then we define some physical facts and a function to randomize them:

In [2]:
const dist = 3 # distance for ball to fly [m]
const vs   = 10 # serve velocity [m/s]
const vr   = 20 # return velocity [m/s]

rd(s::Float64) = randn()*s + 1

rd (generic function with 1 method)

Next we must describe the behaviour of our players. They are modeled as finite state machines, which have known states and react to known events. This is done with the `step!` function. Julia's multiple dispatch allows to give multiple definitions of `step!` for different combinations of states and events.

The `serve` and `ret`-functions, used for describing serving and return of players are used to randomize the time and the behaviour of players. The players thus act probabilistically as Markov automata.

In [3]:
function init!(p::Player, opp::Player)
    p.opp = opp
    if rand() ≤ p.attentiveness
        p.state = Wait()
    else
        p.state = Unalert()
    end
end

function serve(p::Player)
    ts = 3 + dist*rd(0.15)/(vs*rd(0.25))
    if (rand() ≤ p.accuracy) && (p.state == Wait())
        event!(𝐶, SF(step!, p.opp, Serve()), after, ts)
        @printf("%5.2f: %s serves %s\n", tau()+ts, p.name, p.opp.name)
    else
        event!(𝐶, SF(step!, p.opp, Miss()), after, ts)
        @printf("%5.2f: %s serves and misses %s\n", tau()+ts, p.name, p.opp.name)
    end
    if rand() ≥ p.attentiveness
        p.state = Unalert()
    end
end

function ret(p::Player)
    tr = dist*rd(0.15)/(vr*rd(0.25))
    if rand() ≤ p.accuracy
        event!(𝐶, SF(step!, p.opp, Return()), after, tr)
        @printf("%5.2f: %s returns %s\n", tau()+tr, p.name, p.opp.name)
    else
        event!(𝐶, SF(step!, p.opp, Miss()), after, tr)
        @printf("%5.2f: %s returns and misses %s\n", tau()+tr, p.name, p.opp.name)
    end
    if rand() ≥ p.attentiveness
        p.state = Unalert()
    end
end

ret (generic function with 1 method)

The actual behaviour of a player is implemented as a state machine via the `step!`--function.

In [4]:
"default transition for players"
step!(p::Player, q::PState, σ::PEvent) =
        println("undefined transition for $(p.name), $q, $σ")

"player p gets a start command"
step!(p::Player, ::Union{Wait, Unalert}, ::Start) = serve(p)

"player p is waiting and gets served or returned"
step!(p::Player, ::Wait, ::Union{Serve, Return}) = ret(p)

"player p is unalert and gets served or returned"
function step!(p::Player, ::Unalert, ::Union{Serve, Return})
    @printf("%5.2f: %s looses ball\n", τ(), p.name)
    p.opp.score += 1
    p.state = Wait()
    serve(p)
end

"player p is waiting or unalert and gets missed"
function step!(p::Player, ::Union{Wait, Unalert}, ::Miss)
    p.score += 1
    p.state = Wait()
    serve(p)
end

"simplified `step!` call"
step!(p::Player, σ::PEvent) = step!(p, p.state, σ)

step!

In order to setup a simulation, we have to create and initialize the players, to start and run the game:

In [5]:
ping = Player("Ping", 0.90, 0.90)
pong = Player("Pong", 0.90, 0.90)
init!(ping, pong)
init!(pong, ping)
step!(ping, Start())

Random.seed!(123)

println(run!(𝐶, 30))
println("Ping scored $(ping.score)")
println("Pong scored $(pong.score)")

 3.33: Ping serves Pong
 3.33: Pong looses ball
 6.57: Pong serves Ping
 6.74: Ping returns Pong
 6.90: Pong returns Ping
 7.02: Ping returns Pong
 7.20: Pong returns Ping
 7.32: Ping returns Pong
 7.43: Pong returns Ping
 7.54: Ping returns Pong
 7.63: Pong returns Ping
 7.77: Ping returns Pong
 7.91: Pong returns Ping
 8.08: Ping returns Pong
 8.20: Pong returns Ping
 8.34: Ping returns Pong
 8.51: Pong returns Ping
 8.73: Ping returns Pong
 8.92: Pong returns Ping
 9.01: Ping returns Pong
 9.64: Pong returns Ping
 9.79: Ping returns Pong
 9.87: Pong returns Ping
10.02: Ping returns Pong
10.14: Pong returns Ping
10.27: Ping returns Pong
10.47: Pong returns Ping
10.64: Ping returns and misses Pong
13.84: Pong serves and misses Ping
17.12: Ping serves Pong
17.29: Pong returns and misses Ping
20.59: Ping serves Pong
20.73: Pong returns Ping
20.85: Ping returns Pong
20.97: Pong returns Ping
21.25: Ping returns Pong
21.40: Pong returns Ping
21.55: Ping returns Pong
21.65: Pong returns Pin

Finally we reset `𝐶` for further simulations.

In [6]:
reset!(𝐶)

"clock reset to t₀=0.0, sampling rate Δt=0.0."